In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd

In [ ]:
# SIMULATIONS

chunksize = 10 ** 6  # Adjust chunk size based on your memory availability
chunks = []

for chunk in pd.read_csv(r'C:\Users\mcthu\PycharmProjects\MarkovianEmbedding\position_velocity_data.csv', chunksize=chunksize, low_memory=False):
    # process each chunk individually or filter/aggregate it
    chunks.append(chunk)

df = pd.concat(chunks, ignore_index=True)

# Extract only the columns that start with 'Position' or 'Velocity'
position_cols = [col for col in df.columns if col.startswith('Position')]
velocity_cols = [col for col in df.columns if col.startswith('Velocity')]

timestep_sim = df['real_timestep_s'].values[0]
positions_sim = []
velocities_sim = []
for i in range(len(position_cols)):
    positions_sim.append(df[position_cols[i]].astype(float).values)
    velocities_sim.append(df[velocity_cols[i]].astype(float).values)

# print(f"Positions shape: {positions_sim.shape}")
# print(f"Velocities shape: {velocities_sim.shape}")

bin_num_sim = 10
fs_sim = 1/(timestep_sim * bin_num_sim)

def bin_data(series, bin_size):
    # Ensuring the length of series is divisible by bin_size
    length = len(series) - len(series) % bin_size
    series = np.array(series[:length])
    return np.mean(series.reshape(-1, bin_size), axis=1)

traces_sim = []
for trace in positions_sim:
    series = bin_data(trace, bin_num_sim)
    traces_sim.append(series)

v_traces_sim = []
for trace in velocities_sim:
    series = bin_data(trace, bin_num_sim)
    v_traces_sim.append(series)

In [1]:
# SIM PART 2
vos = [-2, -1, 0, 1, 2]
tol = .005
hist_len = int(10e-6/fs_sim)

sim_histories = []


# integrand = v / t**(1.5)

for speed in vos:
    print("length of history is " + str(hist_len) )
    print("length of trace is " + str(len(v_traces_sim[0])))

    for i in range(len(traces_sim)):
        v_tolerance = (speed * np.std(v_traces_sim[i]))
        speed_tolerance = tol * np.std(v_traces_sim[i])
        v_indices = np.where((v_traces_sim[i] < v_tolerance + speed_tolerance) & (v_traces_sim[i] > v_tolerance - speed_tolerance))[0]
        print("speed idx sim: " + str(len(v_indices)))

        hist_idx = []
        for idx in v_indices:
            if idx - hist_len > 0: #idx - last_idx > (init_len / 100) and
                hist_idx.append(idx)

        print("total hist idx: " + str(len(hist_idx)))

        for idx in hist_idx:
            v = v_traces_sim[idx - hist_len: idx][::-1]
            t_hist = [range(1, len(v) + 1)]*(1/fs_sim)
            plt.plot(t_hist, v)



NameError: name 'fs_sim' is not defined